In [1]:
#general imports
import os
import platform
import re
import logging
from pprint import pprint

#spikeinterface imports
import spikeinterface
import spikeinterface.sorters as ss
import spikeinterface.full as si

#local imports
from file_selector import main as file_selector_main
#from AxonReconPipeline.src.archive.extract_raw_assay_data import main as extract_raw_assay_data_main
from generate_recording_and_sorting_objects import main as generate_recording_and_sorting_objects
from axon_trace_classes import axon_trace_objects
from reconstruct_axons import reconstruct_axon_trace_objects
import mea_processing_library as MPL
import helper_functions as helper
import spike_sorting as hp #hornauerp, https://github.com/hornauerp/axon_tracking/blob/main/axon_tracking/spike_sorting.py
import template_extraction as te_hp

#Logger Setup
#Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create handlers
stream_handler = logging.StreamHandler()  # logs to console
#file_handler = logging.FileHandler('file.log')  # logs to a file

# Set level of handlers
stream_handler.setLevel(logging.DEBUG)
#file_handler.setLevel(logging.ERROR)

# Add handlers to the logger
logger.addHandler(stream_handler)
#logger.addHandler(file_handler)

# Create formatters and add it to handlers
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s - %(module)s.%(funcName)s')
stream_handler.setFormatter(formatter)

{'detect_threshold': 0.01,
 'detection_type': 'relative',
 'distance_exp': 2,
 'edge_dist_amp_ratio': 0.3,
 'init_amp_peak_ratio': 0.2,
 'init_delay': 0,
 'kurt_threshold': 0.1,
 'mad_threshold': 8,
 'max_distance_for_edge': 300,
 'max_distance_to_init': 200,
 'max_peak_latency_for_splitting': 0.5,
 'min_outlier_tracking_error': 50,
 'min_path_length': 100,
 'min_path_points': 5,
 'min_points_after_branching': 3,
 'min_selected_points': 30,
 'n_neighbors': 3,
 'neighbor_radius': 100,
 'peak_std_distance': 30,
 'peak_std_threshold': 0.8,
 'r2_threshold': 0.8,
 'r2_threshold_for_outliers': 0.98,
 'remove_isolated': True,
 'split_paths': True,
 'theilsen_maxiter': 10000,
 'upsample': 5}


In [2]:
#import all functions in main.py
from main import *

debug_mode = True
#testing Full Activity Scan and Network Scan...
pre_selected_folders = [        
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/ActivityScan/000023",
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/Network/000024",
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/Tests_Adam/Tests_Adam/240118/M06844/AxonTracking/000026",
    #testing continuity test with non-continuous data
    #"/mnt/disk20tb/PrimaryNeuronData/Maxone/SPTAN1_1/230725/16657/ActivityScan",

    #new comparison 09Feb24
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M05506/AxonTracking/000003"
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M07038/AxonTracking"
    ]
#Organoid effort 31Mar24
pre_selected_folders = [
    #reconstructions not working here...
    #'/mnt/ben-shalom_nas/irc/media/harddrive8tb/IMR90_T3_03262024_AP/IMR90_T3_03262024_AP/240328/16719/AxonTracking/000018'
    #same chip, fuller recording
    #'/mnt/ben-shalom_nas/irc/media/harddrive8tb/IMR90_T3_03262024_AP/IMR90_T3_03262024_AP/240328/16719/AxonTracking/000017',
    
    #Other organoid chips
    #'/mnt/ben-shalom_nas/irc/media/harddrive8tb/IMR90_T3_03262024_AP/IMR90_T3_03262024_AP/240326/18832/AxonTracking/000009',
    
    #Primary neuron Folic Acid
    #new comparison 09Feb24
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M05506/AxonTracking/000003",
    #"/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/FolicAcid_AxonTracking_05022024/FolicAcid_AxonTrack_Comparison/240205/M07038/AxonTracking/000006"

    ]

KCNT1_effort = [
    '/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082'
]
continuous_h5_dirs = select_folders_and_get_continuous_h5_dirs(pre_selected_folders = KCNT1_effort, debug_mode = debug_mode)
pprint(continuous_h5_dirs)

2024-05-07 17:59:00,678 - INFO - Selecting folders to process, checking for continuity, and extracting the .h5 file path: - main.select_folders_and_get_continuous_h5_dirs
2024-05-07 17:59:00,679 - INFO - Selected folders: ['/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082'] - main.select_folders_and_get_continuous_h5_dirs
2024-05-07 17:59:00,680 - INFO - Extracting .h5 file paths from directories: - mea_processing_library.extract_raw_h5_filepaths



*** DEBUG MODE ENABLED in file_selector.py ***
/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082

Folders selected for analysis:


2024-05-07 17:59:15,869 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 17:59:15,870 - INFO - Testing continuity of /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082/data.raw.h5: - mea_processing_library.test_continuity
2024-05-07 17:59:15,871 - INFO - Counting wells and recordings in /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082/data.raw.h5: - mea_processing_library.count_wells_and_recs
2024-05-07 17:59:15,872 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 17:59:15,872 - INFO - Scan Type: AxonTracking - mea_processing_library.count_wells_and_recs
2024-05-07 18:00:10,575 - INFO - MaxTwo Detected. - mea_processing_library.count_wells_and_recs
2024-

Data for /mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082/data.raw.h5 is continuous.
{0: {'chipID': 'M08034',
     'date': '240503',
     'h5_file_path': '/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082/data.raw.h5',
     'runID': '000082',
     'scanType': 'AxonTracking'}}


In [3]:
print(continuous_h5_dirs)

{0: {'h5_file_path': '/mnt/ben-shalom_nas/rbs_maxtwo/rbsmaxtwo/media/rbs-maxtwo/harddisk20tb/KCNT1_T4_C1_04122024/KCNT1_T4_C1_04122024/240503/M08034/AxonTracking/000082/data.raw.h5', 'runID': '000082', 'scanType': 'AxonTracking', 'chipID': 'M08034', 'date': '240503'}}


In [4]:
#Set working directory for following cells to work properly (if running in Jupyter Notebook)
#This directory should be the root of the git repository
working_dir = os.getcwd()
if working_dir not in '/mnt/disk15tb/adam/git_workspace/MEA_Analysis':
    os.chdir('/mnt/disk15tb/adam/git_workspace/MEA_Analysis')
    working_dir = os.getcwd()
print(working_dir)

/mnt/disk15tb/adam/git_workspace/MEA_Analysis


In [5]:
#Set up matplotlib to work properly in Jupyter Notebook
%matplotlib inline

In [6]:

#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan only needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
recordings_dir = './AxonReconPipeline/data/temp_data/recordings'
#only get sortings for well001
stream_select = 0
##2. Generate multirecording objects for each stream for spikesorting and waveform extraction steps
multi_recs_by_scan, common_el_by_scan = generate_multirecordings_by_stream_by_scan(continuous_h5_dirs,
                                                                                    allowed_scan_types,
                                                                                    recordings_dir,
                                                                                    stream_select = stream_select,
                                                                                    )

2024-05-07 19:33:22,358 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 19:33:31,417 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 19:33:31,561 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0000 - spike_sorting.concatenate_recording_slices


"Unable to open object (object 'rec0020' doesn't exist)"


2024-05-07 19:33:31,715 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0001 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:31,855 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0002 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:31,989 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0003 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:32,117 - INFO - Loaded centered recording rec0004 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0004 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:32,254 - INFO - Loaded centered recording rec0005 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000

"Unable to open object (object 'rec0020' doesn't exist)"


In [7]:

# #Plot common electrodes
# recording_dir = './AxonReconPipeline/data/temp_data/recordings'
# #note this automatically saves the plot to the recording_dir for the date/chip/scan/run/stream 
# widgets_by_scan = plot_electrodes(multi_recs_by_scan, recording_dir=recording_dir, auto_plot = False)
# #plt.tight_layoutplot_progressive_recording_channel_exclusion_plots
# #plt.show()

In [8]:
import numpy as np
import spike_sorting as hp

#generate progressive channel exclusion plots
def plot_progressive_recording_channel_exclusion_plots(rec_path, 
                                 stream_id, 
                                 recording_dir = './AxonReconPipeline/data/temp_data/recordings/',
                                 #stream_break = None
                                 ):
    """
    Function that centers and concatenates the recordings of an axon scan for all common electrodes. 

    Arguments
    ----------
    rec_path: str
        Path to the axon scan file.
    stream_id: str
        Well ID in the format "well***"; Well 1 would be "well001", Well 20 would be "well020"
        
    Returns
    ----------
    multirecording: ConcatenatedRecordingSlice
        Concatenated recording across common electrodes (spikeinterface object)
    """
    rec_names, common_el = hp.find_common_electrodes(rec_path, stream_id)
    recording_details = MPL.extract_recording_details(rec_path)
    date = recording_details[0]['date']
    chip_id = recording_details[0]['chipID']
    scanType = recording_details[0]['scanType']
    run_id = recording_details[0]['runID']
    recording_dir_by_stream =recording_dir+f'/{date}/{chip_id}/{scanType}/{run_id}'
    
    rec_list = []
    recording_dir_by_stream = os.path.join(recording_dir_by_stream, stream_id)
    
        
    for rec_name in rec_names: 
        #rec_name = 'rec' + '%0*d' % (4, r)
        rec = si.MaxwellRecordingExtractor(rec_path, stream_id=stream_id, rec_name=rec_name)
                
        ch_id = rec.get_property("contact_vector")['device_channel_indices']
        rec_el = rec.get_property("contact_vector")["electrode"]
        
        chan_idx = [np.where(rec_el == el)[0][0] for el in common_el]
        sel_channels = rec.get_channel_ids()[chan_idx]
        #chunk_size = np.min([10000, rec.get_num_samples()]) - 100 #Fallback for ultra short recordings (too little activity)
        #rec_centered = si.center(rec,chunk_size=chunk_size)
        #rec_list.append(rec_centered)
        rec_wrap = [[rec]]
        #recording_dir = recording_dir
        #note this automatically saves the plot to the recording_dir for the date/chip/scan/run/stream 
        widgets_by_scan = plot_electrodes(rec_wrap, recording_dir=recording_dir, auto_plot = False, name = rec_name)

        #rec_list.append(rec.channel_slice(sel_channels, renamed_channel_ids=list(range(len(chan_idx)))))
        #multirecording = si.concatenate_recordings(rec_list)
        #widgets_by_scan = plot_electrodes([[multirecording]], recording_dir=recording_dir, auto_plot = False, name = f"prog_{rec_name}")
    
    

    # if not os.path.exists(recording_dir_by_stream):
    #     os.makedirs(recording_dir_by_stream)
    # save_kwargs = dict(n_jobs=12)
    # multirecording.save(folder = recording_dir_by_stream, overwrite = True, verbose = True, **save_kwargs)

    # return multirecording, common_el
        
#plot_progressive_recording_channel_exclusion_plots(continuous_h5_dirs[0]['h5_file_path'], "well001", recording_dir = recording_dir)        

In [9]:
#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan only needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
recordings_dir = './AxonReconPipeline/data/temp_data/recordings'
sortings_dir = './AxonReconPipeline/data/temp_data/sortings'
#only get sortings for well001
stream_select = 0
##3. Extract sorting objects per stream from sliced and cocatenated recordings (most/all fixed electrodes during axontracking scan)
sorting_lists_by_scan = get_list_of_sortings_by_scan(continuous_h5_dirs, 
                                                     recordings_dir = recordings_dir,
                                                     sortings_dir = sortings_dir, 
                                                     allowed_scan_types = allowed_scan_types,
                                                     stream_select = stream_select,)

2024-05-07 19:33:35,044 - INFO - Loading and merging the recordings as needed from the .h5 files. Generating spike sorting objects and merging as needed. - main.get_list_of_sortings_by_scan
2024-05-07 19:33:35,045 - INFO - Allowed scan types: ['AxonTracking'] - main.get_list_of_sortings_by_scan
2024-05-07 19:33:35,046 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
Sorting recordings: 100%|██████████| 1/1 [00:00<00:00,  5.09it/s]


In [10]:
# import matplotlib.pyplot as plt
# import spikeinterface.extractors as se

# # Assuming sorting_lists_by_scan[0][0] is a SortingExtractor instance
# sorting = sorting_lists_by_scan[0][0]

# # Get unit ids
# unit_ids = sorting.get_unit_ids()

# # Get unit locations
# unit_locations = [sorting.get_unit_property(unit_id, 'location') for unit_id in unit_ids]

# # Unpack locations
# x_locs = [loc[0] for loc in unit_locations]
# y_locs = [loc[1] for loc in unit_locations]

# # Create plot
# plt.scatter(x_locs, y_locs)
# plt.title('Unit Locations')
# plt.xlabel('X Location')
# plt.ylabel('Y Location')
# plt.show()

In [11]:
import template_extraction as te_hp
import tqdm

def extract_waveform_from_segments(h5_file_path, stream_id, segment_sorting, sel_unit_id, save_root, peak_cutout=2, align_cutout=True, upsample=2, rm_outliers=True, n_jobs=16, n_neighbors=10, overwrite_wf=False, overwrite_tmp = False):
    #full_path = segment_sorting._kwargs['recording_list'][0]._parent_recording._kwargs['recording']._kwargs['file_path']
    try:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._kwargs['parent_recording']._kwargs['recording']._kwargs['file_path']
    except:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._parent_recording._kwargs['folder_path']
        #segment_sorting._kwargs['recording_list'][0]._kwargs['parent_recording']._kwargs['folder_path']
        #get parent folder instead of file path
        full_path = os.path.dirname(full_path)
    
    if save_root is None:
        #save_root = os.path.dirname(full_path) 
        recording_details = MPL.extract_recording_details(full_path)
        date = recording_details[0]['date']
        chip_id = recording_details[0]['chipID']
        scanType = recording_details[0]['scanType']
        run_id = recording_details[0]['runID']
        save_root = f'./AxonReconPipeline/data/temp_data/waveforms/{date}/{chip_id}/{scanType}/{run_id}/{stream_id}'
    waveforms = te_hp.extract_waveforms(h5_file_path, segment_sorting, stream_id, save_root, n_jobs, overwrite_wf)
    return waveforms

def extract_all_waveforms(h5_file_path, stream_id, segment_sorting, pos, te_params, save_root=None):
    sel_unit_ids = segment_sorting.get_unit_ids()

    try:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._kwargs['parent_recording']._kwargs['recording']._kwargs['file_path']
    except:
        full_path = segment_sorting._kwargs['recording_or_recording_list'][0]._parent_recording._kwargs['folder_path']
        #segment_sorting._kwargs['recording_list'][0]._kwargs['parent_recording']._kwargs['folder_path']
        #get parent folder instead of file path
        full_path = os.path.dirname(full_path)
    
    if save_root is None:
        #save_root = os.path.dirname(full_path) 
        recording_details = MPL.extract_recording_details(full_path)
        date = recording_details[0]['date']
        chip_id = recording_details[0]['chipID']
        scanType = recording_details[0]['scanType']
        run_id = recording_details[0]['runID']
        save_root = f'./AxonReconPipeline/data/temp_data/waveforms/{date}/{chip_id}/{scanType}/{run_id}/{stream_id}'
    waveform_save_path = os.path.join(save_root, 'waveforms')
    if not os.path.exists(waveform_save_path):
        os.makedirs(waveform_save_path)
        
    for sel_unit_id in sel_unit_ids: 
        #template_save_file = os.path.join(template_save_path, str(sel_unit_id) + '.npy')
                ## debug
        # if sel_unit_id != 203: 
        # ##
        break
        
    if te_params['overwrite_tmp']:
        try:
            extract_waveform_from_segments(h5_file_path, stream_id, segment_sorting, sel_unit_id, save_root, **te_params)
            #grid = convert_to_grid(template_matrix, pos)
            #np.save(template_save_file, merged_template)
        except Exception as e:
            print(f'{e}')

def extract_waveforms_from_sorting_dict(sorting_list, h5_file_path, qc_params={}, te_params={}, stream_break = None):
    #rec_list = list(sorting_dict.keys())
    
    rec_list = h5_file_path
    if isinstance(h5_file_path, str):
        rec_list = [h5_file_path]    

    for rec_path in rec_list:
        #sorting_list = sorting_dict[rec_path]
        #sorting_list = sorting_dict
        #sorting_list.sort()

        for sorting in sorting_list:
        #for sorting_path in sorting_list:
            #sorting = si.KiloSortSortingExtractor(sorting_path)
            sorting_path = sorting._kwargs['folder_path']
            # #Temporary debug
            # #if 'Network' in stream_id, replace with 'AxonTracking'
            # if 'Network' in sorting_path:
            #     sorting_path = sorting_path.replace('Network', 'AxonTracking')
            stream_id = [p for p in sorting_path.split('/') if p.startswith('well')][0] #Find out which well this belongs to
            
            # print(stream_id)
            # #debug:
            # stream_number = int(stream_id[-1])
            # if stream_number > 0:
            #     continue
            # #debug end
            
            #rec_names, common_el, pos = ss.find_common_electrodes(rec_path, stream_id)
            multirecording, pos = hp.concatenate_recording_slices(rec_path, stream_id)

            #duration = int(h5['assay']['inputs']['record_time'][0].decode('UTF-8')) * n_recs #In case we want to use firing rate as criterion
            cleaned_sorting = te_hp.select_units(sorting, **qc_params)
            cleaned_sorting = si.remove_excess_spikes(cleaned_sorting, multirecording) #Relevant if last spike time == recording_length
            cleaned_sorting.register_recording(multirecording)
            segment_sorting = si.SplitSegmentSorting(cleaned_sorting, multirecording)
            extract_all_waveforms(h5_file_path, stream_id, segment_sorting, pos, te_params, save_root=None)

            if stream_break is not None:
                formatted = 'well{:03}'.format(stream_break)
                if stream_id == stream_break:
                    break

te_params = dict()
te_params['align_cutout'] = True #Align waveforms by max waveform peak
te_params['upsample'] = 2 #Factor by which to upsample waveforms
te_params['rm_outliers'] = True #Check if outliers should be removed
te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
te_params['n_neighbors'] = 10 #Number of neighbors for outlier detection
te_params['peak_cutout'] = 2 #Looking for peak +- this value around the expected peak (removing minor offsets)
te_params['overwrite_wf'] = False #Flag if waveform extraction should be repeated (e.g. different cutouts)
te_params['overwrite_tmp'] = True #Flag if templates should be recalculated if already existing

qc_params = dict()
qc_params['min_n_spikes'] = 500 #Minimum number of spikes to be detected for a unit for template extraction to take place
qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort

for i, sorting_list_by_stream in enumerate(sorting_lists_by_scan):
    extract_waveforms_from_sorting_dict(sorting_list_by_stream, continuous_h5_dirs[i]['h5_file_path'], qc_params=qc_params, te_params=te_params, stream_break = None)

2024-05-07 19:33:45,239 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details


"Unable to open object (object 'rec0020' doesn't exist)"


2024-05-07 19:33:45,586 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0000 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:46,199 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0001 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:46,604 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0002 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:47,450 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0003 - spike_sorting.concatenate_recording_slices
2024-05-07 19:33:48,063 - INFO - Loaded centered recording rec0004 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000

"Unable to open object (object 'rec0020' doesn't exist)"


2024-05-07 19:33:57,010 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 19:34:06,098 - INFO - Waveforms loaded from /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240503/M08034/AxonTracking/000082/well000/waveforms/seg0 already exist - template_extraction.extract_waveforms
2024-05-07 19:34:11,668 - INFO - Waveforms loaded from /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240503/M08034/AxonTracking/000082/well000/waveforms/seg1 already exist - template_extraction.extract_waveforms
2024-05-07 19:34:17,860 - INFO - Waveforms loaded from /mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/waveforms/240503/M08034/AxonTracking/000082/well000/waveforms/seg2 already exist - template_extraction.extract_waveforms
2024-05-07 19:34:23,133 - INFO - Waveforms loaded from /mnt/disk15tb/adam/git_workspace/MEA_Analysis/

"Unable to open object (object 'rec0020' doesn't exist)"


In [12]:
#Set allowed scan types, axontracking alone is currently the best option
allowed_scan_types = [
    #"ActivityScan", Dense activity scan gonly needed to define locations of electrodes in network and axon tracking scans
    "AxonTracking", 
    #"Network"
    ]
templates_dir='./AxonReconPipeline/data/temp_data/templates'  
##3. Extract Templates
#Extract recording details from the .h5 file path
#only get tempaltes for well000
stream_select = 0
just_load_waveforms = True
templates_by_scan = get_merged_templates_by_unit_by_scan(continuous_h5_dirs, 
                                                         sorting_lists_by_scan,
                                                         allowed_scan_types=allowed_scan_types,
                                                         templates_dir = templates_dir,
                                                         stream_select = stream_select, 
                                                         just_load_waveforms = just_load_waveforms)

2024-05-07 19:35:55,299 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
2024-05-07 19:35:55,353 - INFO - Loaded centered recording rec0000 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0000 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,415 - INFO - Loaded centered recording rec0001 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0001 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,485 - INFO - Loaded centered recording rec0002 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0002 - spike_sorting.concatenate_recording_slices


"Unable to open object (object 'rec0020' doesn't exist)"


2024-05-07 19:35:55,547 - INFO - Loaded centered recording rec0003 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0003 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,609 - INFO - Loaded centered recording rec0004 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0004 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,673 - INFO - Loaded centered recording rec0005 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0005 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,737 - INFO - Loaded centered recording rec0006 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000082/well000/rec0006 - spike_sorting.concatenate_recording_slices
2024-05-07 19:35:55,800 - INFO - Loaded centered recording rec0007 from ./AxonReconPipeline/data/temp_data/recordings/240503/M08034/AxonTracking/000

"Unable to open object (object 'rec0020' doesn't exist)"


2024-05-07 19:35:56,809 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details
  0%|          | 0/227 [00:00<?, ?it/s]2024-05-07 19:35:58,892 - INFO - Partial template saved to ./AxonReconPipeline/data/temp_data/templates/240503/M08034/AxonTracking/000082/well000/partial_templates/seg0_unit10.npy - template_extraction.combine_templates
2024-05-07 19:36:00,962 - INFO - Partial template saved to ./AxonReconPipeline/data/temp_data/templates/240503/M08034/AxonTracking/000082/well000/partial_templates/seg1_unit10.npy - template_extraction.combine_templates
2024-05-07 19:36:03,471 - INFO - Partial template saved to ./AxonReconPipeline/data/temp_data/templates/240503/M08034/AxonTracking/000082/well000/partial_templates/seg2_unit10.npy - template_extraction.combine_templates
2024-05-07 19:36:06,113 - INFO - Partial template saved to ./AxonReconPipeline/data/temp_data/templates/240503/M08034/AxonTracking/000082/well000/partial_templates/seg

...

Merging matching channels...
Total merged channels: 2475
Total channels: 2475
Processing 1005 channels
Processing 268 matching channels
Processing matching channels across footprints, template 4/4...
Processing 737 non-matching channels


Processing non-matching channels in each footprint, template 4/4: 100%|██████████| 65/65 [00:00<00:00, 233.52it/s]


done merging
Total merged channels: 3212
Filling in missing channels...
Processing 22948 unrecorded channels


2024-05-07 22:53:12,786 - INFO - Merged template saved to ./AxonReconPipeline/data/temp_data/templates/240503/M08034/AxonTracking/000082/well000/merged_templates - template_extraction.extract_all_templates
100%|██████████| 227/227 [3:17:15<00:00, 52.14s/it]

Total merged channels after fill: 26160


In [13]:


from IPython.core.debugger import set_trace
from pathlib import Path
import shutil

### Initiate Axon trace algo
import numpy as np
import matplotlib.pylab as plt
import MEAutility as mu
from scipy.signal import resample_poly
from scipy.stats import kurtosis, linregress
from matplotlib import gridspec
from scipy import io
import numpy as np
import networkx as nx
from pathlib import Path
import sys
from pprint import pprint
from axon_velocity import *
from axon_velocity.models import load_cell
from axon_velocity.evaluation import *
import os
from pathlib import Path
import pickle
import re
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from probeinterface import plotting as plotting_probe
import json

import concurrent.futures
from tqdm import tqdm
from multiprocessing import Manager

#for testing
import numpy as np
from shapely.geometry import Polygon
import itertools
from shapely.geometry import Point
from shapely.geometry import Polygon
from collections import Counter
import math
from collections import defaultdict
import mea_processing_library as MPL

#import all functions in main.py
from main import *

#Define Algorithm Params
params = get_default_graph_velocity_params()
options = ['axon_velocity', 'organoid']
option = options[0]
if option == 'axon_velocity':
    # change params (fig6 Buccino axon_velocity paper example)
    params['detect_threshold'] = 0.01
    params['kurt_threshold'] = 0.1
    params['peak_std_threshold'] = 0.8
    params['upsample'] = 5
    params['neighbor_radius'] = 100
    params['r2_threshold'] = 0.8

elif option == 'organoid':
    ## Modulating params for organoid data

    # General
    params['upsample'] = 5 #1  # Upsample waveforms by this factor, upsample = 1 means no upsampling
    params['min_selected_points'] = 20 #30  # Minimum number of points to be selected for graph propagation, typically 30
    params['verbose'] = True  # Verbose output

    # Channel selection
    params['detect_threshold'] = 0.01  # Detection threshold (with respect to channel featuring maximal signal)
    params['detection_type'] = 'relative'  # Whether to use an 'absolute' or 'relative' detection threshold
    params['kurt_threshold'] = 0.05 #0.3  # Kurtosis threshold below which a channel is discarded
    params['peak_std_threshold'] = 0.5 #1  # Peak time standard deviation threshold in ms below which a channel is discarded
    params['init_delay'] = 0 #0.1  # Initial delay in seconds (with respect to maximum channel) below which a channel is discarded
    params['peak_std_distance'] = 50 #30  # Distance in µm to select channel neighborhood to compute peak time standard deviation
    params['remove_isolated'] = False #True  # If True, isolated channels are removed from selection

    # Graph
    params['init_amp_peak_ratio'] = 0.2  # Scalar value that weighs the contribution of the amplitude and the peak latency for h_init (ω_init in equation (2))
    params['max_distance_for_edge'] = 200 #100  # Maximum distance in µm between channels to create a graph edge
    params['max_distance_to_init'] = 500 #200  # Maximum distance in µm between a channel and the init_channel to create a graph edge
    params['n_neighbors'] = 5 #3  # Maximum number of edges that one channel can connect to below which an axonal branch is discarded
    params['distance_exp'] = 2  # Exponent for distance computation (e in equation (3))
    params['edge_dist_amp_ratio'] = 0.5 #0.3  # Relative weight between distance and amplitude to select neighbor nodes for graph edges

    # Axonal reconstruction
    params['min_path_length'] = 50 #100  # Minimum axon path length in µm to include an axonal branch
    params['min_path_points'] = 4 #5  # Minimum number of channels in an axon path to include an axonal branch
    params['neighbor_radius'] = 150 #100  # Radius in µm to exclude neighboring channels around an identified path
    params['min_points_after_branching'] = 3  # Minimum number of points after a branching to avoid pruning

    # Path cleaning/velocity estimation
    params['mad_threshold'] = 8  # Threshold in median absolute deviations on the fit error to consider points as outliers in the velocity estimation
    params['split_paths'] = True  # If True, the final path splitting step is enabled
    params['max_peak_latency_for_splitting'] = 0.5  # If a jump in the peak latencies of a path exceeds this value, the path can be split in sub-paths
    params['r2_threshold'] = 0.7 #0.9  # R^2 threshold for velocity linear fit below which an axon branch is discarded
    params['r2_threshold_for_outliers'] = 0.9 #0.98  # R^2 threshold below which outliers are detected and removed
    params['min_outlier_tracking_error'] = 75 #50  # Tracking error in µm above which a point can be considered an outlier and removed


In [14]:

def get_extremum(template, locations):
    """
    Get the extremum of the template.
    """
    # Get the extremum index along the first dim of the template, aligning with channel locations
    extremum_idx = np.unravel_index(np.argmax(np.abs(template)), template.shape)[0]
    extremum = locations[extremum_idx]
    return extremum


stream_select = 0
templates_dir='./AxonReconPipeline/data/temp_data/templates'
for i, continuous_h5_dir in continuous_h5_dirs.items():
    recording_details = MPL.extract_recording_details(continuous_h5_dir['h5_file_path'])
    date = recording_details[0]['date']
    chip_id = recording_details[0]['chipID']
    scanType = recording_details[0]['scanType']
    run_id = recording_details[0]['runID']
    h5 = h5py.File(continuous_h5_dir['h5_file_path'], 'r')
    stream_ids = list(h5['wells'].keys())
    for stream_id in stream_ids:
        if stream_select is not None:
            if stream_id != f'well{stream_select:03}':
                continue
            else:
                print(f'Processing stream {stream_id}')
                break

    temp_dir = os.path.dirname(templates_dir)
    sorting_dir = Path(temp_dir) /'sortings'/ date / chip_id / scanType / run_id / stream_id / 'sorter_output'
    sorting = ss.Kilosort2_5Sorter._get_result_from_folder(sorting_dir)
    recon_dir = Path(temp_dir) /'reconstructions'/ date / chip_id / scanType / run_id / stream_id
    successful_recons = {}  
    unit_ids = sorting.get_unit_ids()
    successful_recons[str(recon_dir)] = {}
    successful_recons[str(recon_dir)]["successful_units"] = {}
    units = []
    branch_ids = []
    velocities = []
    path_lengths = []
    r2s = []
    extremums =[]
    for unit_id in unit_ids:

        print(f'\nProcessing unit {unit_id}')
        # debug
        #if unit_id != 51: continue
        #
                
        merged_template_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}.npy'
        merged_channel_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}_channels.npy'
        merged_template_filled_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}_filled.npy'
        merged_channel_filled_dir = Path(templates_dir) / date / chip_id / scanType / run_id / stream_id / 'templates' / f'{unit_id}_channels_filled.npy'
        try:              

            merged_template = np.load(merged_template_dir)
            merged_channel_loc = np.load(merged_channel_dir)
            merged_template_filled = np.load(merged_template_filled_dir)
            merged_channel_filled_loc = np.load(merged_channel_filled_dir)

            #merged_template = np.load('/mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/templates/240118/M06844/AxonTracking/000026/well001/partial_templates/seg3_unit17.npy')
            #merged_channel_loc = np.load('/mnt/disk15tb/adam/git_workspace/MEA_Analysis/AxonReconPipeline/data/temp_data/templates/240118/M06844/AxonTracking/000026/well001/partial_templates/seg3_unit17_channels.npy')
        except:
            print(f'No merged template found for {unit_id}')
            continue

        ## Channels
        template = merged_template[-1]
        transformed_template = template.T
        #mirror matrix about x-axis to match maxwell orientation
        #transformed_template = np.flip(transformed_template, axis=0)
        
        template_filled = merged_template_filled[-1]
        transformed_template_filled = template_filled.T
        #mirror matrix about x-axis to match maxwell orientation\
        #transformed_template_filled = np.flip(transformed_template_filled, axis=0)
        
        ## Locations
        trans_loc = merged_channel_loc[-1]
        #multiply all y values by -1 to match maxwell orientation
        trans_loc = np.array([[loc[0], loc[1]*-1] for loc in trans_loc])

        trans_loc_filled = merged_channel_filled_loc[-1]
        #multiply all y values by -1 to match maxwell orientation
        trans_loc_filled = np.array([[loc[0], loc[1]*-1] for loc in trans_loc_filled])
        #mirror matrix about x-axis to match maxwell orientation
        #trans_loc_filled = np.flip(trans_loc_filled, axis = 0)

        ##
        gtrs = {}
        fs = 10000
        plot_dir = Path(recon_dir) / f'unit_{unit_id}' / 'axon_tracking_plots'
        if os.path.exists(plot_dir):
            #print(f'Plot directory already exists for unit {unit_id}, skipping...')
            #continue
            shutil.rmtree(plot_dir)
        key = 0

        # Generate amplitude_map figure
        try:
            fig_amp = plt.figure(figsize=(10, 5))
            ax_amp = fig_amp.add_subplot(111)
            #set_trace()
            ax_amp = plot_amplitude_map(transformed_template_filled, 
                                        trans_loc_filled, 
                                        log=False, 
                                        ax=ax_amp, 
                                        cmap="PRGn", 
                                        colorbar=True, 
                                        colorbar_orientation="horizontal")
            ax_amp.set_title(f"Amplitude", fontsize=20)

            #Save figure
            fig_name = "amplitude_map.png"
            fig_path = plot_dir / fig_name
            if not os.path.exists(plot_dir):
                os.makedirs(plot_dir)
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close()

            #accept_units_amp_map.append(key)
        except Exception as e:
            #reject_units_amp_map.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate amplitude_map, error: {e}")

        # Generate peak latency map figure
        try:
            fig_peaks = plt.figure(figsize=(10, 5))
            ax_peaks = fig_peaks.add_subplot(111)
            ax_peaks = plot_peak_latency_map(transformed_template_filled, 
                                             trans_loc_filled, fs=fs, log=False, 
                                             ax=ax_peaks, colorbar=True, 
                                             colorbar_orientation="horizontal")
            ax_peaks.set_title(f"Peak latency", fontsize=20)

            #Save figure
            fig_name = "peak_latency_map.png"
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close()

            #accept_units_peak_latency_map.append(key)
        except Exception as e:
            #reject_units_peak_latency_map.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate peak_latency_map, error: {e}")

        # Select channels and generate selected channels figures
        def plot_selected_channels(fig_title, selected_channels, locations, filename):
            fig = plt.figure(figsize=(10, 5))
            ax = fig.add_subplot(111)
            ax.set_title(fig_title, fontsize=20)
            ax.plot(locations[:, 0], locations[:, 1], marker=".", color="grey", ls="", alpha=0.2)
            ax.plot(locations[selected_channels, 0], 
                    locations[selected_channels, 1], marker=".", color="k", ls="", alpha=0.5)
            ax.axis("off")
            fig_name = filename
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close()
        try:
            # Build axon tracking object
            verbose_bool = True
            gtr0 = None #initialize gtr0
            gtr0 = GraphAxonTracking(transformed_template, trans_loc, fs, 
                                     #verbose=verbose_bool, 
                                     **params)           

            # Select channels
            gtr0.select_channels()                

            # Use the function for each block
            plot_selected_channels(f"Selected after detection threshold: {gtr0._detect_threshold} ms", 
                                np.array(list(gtr0._selected_channels_detect)), gtr0.locations, "detection_threshold.png")
            plot_selected_channels(f"Selected after detection threshold: {gtr0._kurt_threshold} ms", 
                                np.array(list(gtr0._selected_channels_kurt)), gtr0.locations, "kurt_threshold.png")
            plot_selected_channels(f"Selected after peak std threshold: {gtr0._peak_std_threhsold} ms", 
                                    np.array(list(gtr0._selected_channels_peakstd)), gtr0.locations, "peak_std_threshold.png")
            plot_selected_channels(f"Selected after init delay threshold: {gtr0._init_delay} ms", 
                                    np.array(list(gtr0._selected_channels_init)), gtr0.locations, "init_delay_threshold.png")
            plot_selected_channels("Selected after all thresholds", gtr0.selected_channels, gtr0.locations, "all_thresholds.png")

            #if all thresholds are passed, compute graph propagation velocity
            try:
                gtr0 = compute_graph_propagation_velocity(transformed_template, trans_loc, fs, 
                    #verbose=False, 
                    **params)
            except Exception as e:
                print(f"unit {unit_id} failed to compute_graph_propagation_velocity, error: {e}")
            #accept_units_selected_channels.append(key)
        except Exception as e:
            #reject_units_selected_channels.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate selected_channels, error: {e}")              
        
        # Generate axon analytics
        try:

            for i, br in enumerate(gtr0.branches):
                path = br["channels"]
                velocity = br["velocity"]
                r2 = br["r2"]
                length = gtr0.compute_path_length(path)
                units.append(unit_id)
                branch_ids.append(i)
                velocities.append(velocity)
                path_lengths.append(length)
                r2s.append(r2)
                extremums.append(get_extremum(transformed_template, trans_loc))

            #accept_units_axon_reconstruction_heuristics.append(key)
        except Exception as e:
            #reject_units_axon_reconstruction_heuristics.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate axon_analytics, error: {e}")

        # Generate axon_reconstruction_heuristics figure
        try:
            gtr0.build_graph()
            gtr0.find_paths()
            gtr0._verbose = 1

            fig_graph = plt.figure(figsize=(10, 7))
            fig_graph = gtr0.plot_graph(node_search_labels=False, fig=fig_graph, cmap_nodes="viridis", cmap_edges="YlGn")

            fig_name = "axon_reconstruction_heuristics.png"
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"unit {unit_id} successfully generated axon_reconstruction_heuristics")

            #accept_units_axon_reconstruction_heuristics.append(key)
        except Exception as e:
            #reject_units_axon_reconstruction_heuristics.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate axon_reconstruction_heuristics, error: {e}")

        # # Generate axon_reconstruction figures
        # try:
        #     fig_graph = plt.figure(figsize=(12, 7))
        #     #ax_raw = fig_graph.add_subplot(111)                
        #     #axpaths_raw = ax_raw
        #     axpaths_raw = gtr0.plot_branches(cmap="tab20", fig = fig_graph,
        #                                      #plot_bp=True, plot_neighbors=True, 
        #                                         #plot_full_template=True, 
        #                                      #ax=axpaths_raw
        #                                      )
        #     #axpaths_raw.legend(fontsize=6)
        #     #axpaths_raw.set_title("All Branches")

        #     fig_name = "axon_reconstruction_all.png"
        #     fig_path = plot_dir / fig_name
        #     plt.savefig(fig_path, dpi=600, bbox_inches='tight')
        #     plt.close()
            
        #     successful_recons[str(recon_dir)]["successful_units"][str(unit_id)] = {}
        #     #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["stream_id"] = stream_id
        #     #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["fig_path"] = fig_path
        #     print(f"unit {unit_id} successfully generated axon_reconstruction")
        #     #successful_recons[continuous_h5_dir]["template"] = template
        #     #successful_recons[continuous_h5_dir]["merged_channel_loc"] = merged_channel_loc
        #     # gtrs[key] = compute_graph_propagation_velocity(template.T, merged_channel_loc[i], fs, 
        #     #                                         verbose=False, **params)

        #     #accept_units_axon_reconstruction.append(key)
        #     #flag = True
        #     #break
        # except Exception as e:
        #     #reject_units_axon_reconstruction.append(key)
        #     plt.close()
        #     print(f"unit {unit_id} failed to generate all axon_reconstruction, error: {e}")
        
        # Generate axon_reconstruction figures
        try:
            fig_graph = plt.figure(figsize=(12, 7))
            ax_raw = fig_graph.add_subplot(111)                
            axpaths_raw = ax_raw
            axpaths_raw = gtr0.plot_clean_branches(cmap="tab20", plot_bp=True, 
                                                #plot_neighbors=True, 
                                                plot_full_template=True, ax=axpaths_raw)
            axpaths_raw.legend(fontsize=6)
            axpaths_raw.set_title("Clean Branches")

            fig_name = "axon_reconstruction_clean.png"
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=600, bbox_inches='tight')
            plt.close()
            
            successful_recons[str(recon_dir)]["successful_units"][str(unit_id)] = {}
            #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["stream_id"] = stream_id
            #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["fig_path"] = fig_path
            print(f"unit {unit_id} successfully generated clean axon_reconstruction")
            #successful_recons[continuous_h5_dir]["template"] = template
            #successful_recons[continuous_h5_dir]["merged_channel_loc"] = merged_channel_loc
            # gtrs[key] = compute_graph_propagation_velocity(template.T, merged_channel_loc[i], fs, 
            #                                         verbose=False, **params)

            #accept_units_axon_reconstruction.append(key)
            #flag = True
            #break
        except Exception as e:
            #reject_units_axon_reconstruction.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate clean axon_reconstruction, error: {e}")

        # Generate axon_reconstruction figures
        try:
            fig_graph = plt.figure(figsize=(12, 7))
            ax_raw = fig_graph.add_subplot(111)                
            axpaths_raw = ax_raw
            axpaths_raw = gtr0.plot_raw_branches(cmap="tab20", plot_bp=True, plot_neighbors=True, 
                                                plot_full_template=True, ax=axpaths_raw)
            axpaths_raw.legend(fontsize=6)
            axpaths_raw.set_title("Raw Branches")

            fig_name = "axon_reconstruction_raw.png"
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=600, bbox_inches='tight')
            plt.close()
            
            successful_recons[str(recon_dir)]["successful_units"][str(unit_id)] = {}
            #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["stream_id"] = stream_id
            #successful_recons[str(recon_dir)]["successful_units"][str(unit_id)]["fig_path"] = fig_path
            print(f"unit {unit_id} successfully generated axon_reconstruction")
            #successful_recons[continuous_h5_dir]["template"] = template
            #successful_recons[continuous_h5_dir]["merged_channel_loc"] = merged_channel_loc
            # gtrs[key] = compute_graph_propagation_velocity(template.T, merged_channel_loc[i], fs, 
            #                                         verbose=False, **params)

            #accept_units_axon_reconstruction.append(key)
            #flag = True
            #break
        except Exception as e:
            #reject_units_axon_reconstruction.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate raw axon_reconstruction, error: {e}")
            
        # Generate axon_reconstruction_velocities figure
        try:
            fvel, ax_vel = plt.subplots(figsize=(9, 15))
            ax_vel.axis("off")
            fvel = gtr0.plot_velocities(cmap="tab20", plot_outliers=True, fig=fvel, markersize=12, markersize_out=18, fs=20)
            plt.xticks(fontsize=16)
            plt.yticks(fontsize=16)

            fig_name = "axon_reconstruction_velocities.png"
            fig_path = plot_dir / fig_name
            plt.savefig(fig_path, dpi=600, bbox_inches='tight')
            plt.close()
            print(f"unit {unit_id} successfully generated axon_reconstruction_velocities")

            #accept_units_axon_reconstruction_velocities.append(key)
        except Exception as e:
            #reject_units_axon_reconstruction_velocities.append(key)
            plt.close()
            print(f"unit {unit_id} failed to generate axon_reconstruction_velocities, error: {e}")

    import pandas as pd
    df_mea1k = pd.DataFrame({"unit_ids": units, "unit location": extremums, "branch_id": branch_ids, "velocity": velocities,
                         "length": path_lengths, "r2": r2s})
    df_mea1k.to_csv(Path(recon_dir) / "axon_analytics.csv", index=False)
    
    # def convert_keys_to_int(d):
    #     if isinstance(d, dict):
    #         return {int(k) if isinstance(k, np.int64) else k: convert_keys_to_int(v) for k, v in d.items()}
    #     elif isinstance(d, list):
    #         return [convert_keys_to_int(v) for v in d]
    #     else:
    #         return d
    with open(f'{recon_dir}/successful_recons.json', 'w') as f:
        json.dump(successful_recons, f)

2024-05-07 22:53:13,127 - INFO - Extracting recording details from h5 directories: - mea_processing_library.extract_recording_details


Processing stream well000

Processing unit 0
No merged template found for 0

Processing unit 1
No merged template found for 1

Processing unit 2
No merged template found for 2

Processing unit 3
No merged template found for 3

Processing unit 4
No merged template found for 4

Processing unit 5
No merged template found for 5

Processing unit 6
No merged template found for 6

Processing unit 8
No merged template found for 8

Processing unit 9
No merged template found for 9

Processing unit 10
unit 10 successfully generated axon_reconstruction_heuristics
unit 10 successfully generated clean axon_reconstruction
unit 10 successfully generated axon_reconstruction
unit 10 successfully generated axon_reconstruction_velocities

Processing unit 11
No merged template found for 11

Processing unit 12
No merged template found for 12

Processing unit 14
unit 14 failed to compute_graph_propagation_velocity, error: No branches found
unit 14 failed to generate axon_analytics, error: 'NoneType' object i

/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


unit 48 successfully generated axon_reconstruction_heuristics
unit 48 successfully generated clean axon_reconstruction
unit 48 successfully generated axon_reconstruction
unit 48 successfully generated axon_reconstruction_velocities

Processing unit 49
No merged template found for 49

Processing unit 50
No merged template found for 50

Processing unit 51
unit 51 failed to compute_graph_propagation_velocity, error: No branches found
unit 51 failed to generate axon_analytics, error: 'NoneType' object is not iterable
unit 51 failed to generate axon_reconstruction_heuristics, error: No branches found
unit 51 failed to generate clean axon_reconstruction, error: 'NoneType' object is not iterable
unit 51 failed to generate raw axon_reconstruction, error: 'NoneType' object is not iterable
unit 51 failed to generate axon_reconstruction_velocities, error: 'NoneType' object is not iterable

Processing unit 52
No merged template found for 52

Processing unit 54
unit 54 failed to compute_graph_propa

/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/axon_velocity/tracking_classes.py:1217: RuntimeWarning: invalid value encountered in divide
  dist_neighbors_norm = (distance_neighbors - np.min(distance_neighbors)) / np.ptp(distance_neighbors)
/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/axon_velocity/tracking_classes.py:1217: RuntimeWarning: invalid value encountered in divide
  dist_neighbors_norm = (distance_neighbors - np.min(distance_neighbors)) / np.ptp(distance_neighbors)


...

unit 692 failed to compute_graph_propagation_velocity, error: No branches left after cleaning
unit 692 failed to generate axon_analytics, error: 'NoneType' object is not iterable
unit 692 successfully generated axon_reconstruction_heuristics
unit 692 failed to generate clean axon_reconstruction, error: 'NoneType' object is not iterable
unit 692 successfully generated axon_reconstruction
unit 692 failed to generate axon_reconstruction_velocities, error: 'NoneType' object is not iterable

Processing unit 693
unit 693 failed to compute_graph_propagation_velocity, error: Not enough channels selected to compute velocity
unit 693 failed to generate axon_analytics, error: 'NoneType' object is not iterable
unit 693 failed to generate axon_reconstruction_heuristics, error: Not enough channels selected to compute velocity
unit 693 failed to generate clean axon_reconstruction, error: 'NoneType' object is not iterable
unit 693 failed to generate raw axon_reconstruction, error: 'NoneType' object is

/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


unit 714 successfully generated axon_reconstruction_heuristics
unit 714 successfully generated clean axon_reconstruction
unit 714 successfully generated axon_reconstruction
unit 714 successfully generated axon_reconstruction_velocities

Processing unit 715
unit 715 successfully generated axon_reconstruction_heuristics
unit 715 successfully generated clean axon_reconstruction
unit 715 successfully generated axon_reconstruction
unit 715 successfully generated axon_reconstruction_velocities

Processing unit 716
unit 716 failed to compute_graph_propagation_velocity, error: No branches found
unit 716 failed to generate axon_analytics, error: 'NoneType' object is not iterable
unit 716 failed to generate axon_reconstruction_heuristics, error: No branches found
unit 716 failed to generate clean axon_reconstruction, error: 'NoneType' object is not iterable
unit 716 failed to generate raw axon_reconstruction, error: 'NoneType' object is not iterable
unit 716 failed to generate axon_reconstructio

/home/adam/miniconda3/envs/mycondaenv/lib/python3.11/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 2000 reached in spatial median for TheilSen regressor.
  warnings.warn(


unit 718 failed to compute_graph_propagation_velocity, error: No branches left after cleaning
unit 718 failed to generate axon_analytics, error: 'NoneType' object is not iterable
unit 718 successfully generated axon_reconstruction_heuristics
unit 718 failed to generate clean axon_reconstruction, error: 'NoneType' object is not iterable
unit 718 successfully generated axon_reconstruction
unit 718 failed to generate axon_reconstruction_velocities, error: 'NoneType' object is not iterable

Processing unit 719
unit 719 successfully generated axon_reconstruction_heuristics
unit 719 successfully generated clean axon_reconstruction
unit 719 successfully generated axon_reconstruction
unit 719 successfully generated axon_reconstruction_velocities

Processing unit 720
unit 720 successfully generated axon_reconstruction_heuristics
unit 720 successfully generated clean axon_reconstruction
unit 720 successfully generated axon_reconstruction
unit 720 successfully generated axon_reconstruction_veloci